In [1]:
import pandas as pd
from glob import glob
file_lists = glob('C:/python_src/machine_learning/감성분석/data/movie/train/neg/*.txt')
pd_lists = []
for file_path in file_lists[:500]:
    with open(file_path,'r',encoding='utf-8') as f:
        data = {
            'review' : f.read(),
            'target' : 0
        }
        df = pd.DataFrame([data])
        pd_lists.append(df)
train_neg_df = pd.concat(pd_lists,ignore_index=True)
train_neg_df.head()

,review,target
0,Story of a man who has unnatural feelings for ...,0
1,OK its not the best film I've ever seen but at...,0
2,"Amateur, no budget films can be surprisingly g...",0
3,My girlfriend once brought around The Zombie C...,0
4,"Without wishing to be a killjoy, Brad Sykes is...",0


In [2]:
# postive  동일하게   train_pos_df
# train_df = pd.concat([train_nag_df, train_pos_df])
# movie_data.csv로 저장

file_lists = glob('C:/python_src/machine_learning/감성분석/data/movie/train/neg/*.txt')
pd_lists = []
for file_path in file_lists[:500]:
    with open(file_path,'r',encoding='utf-8') as f:
        data = {
            'review' : f.read(),
            'target' : 1
        }
        df = pd.DataFrame([data])
        pd_lists.append(df)
train_pos_df = pd.concat(pd_lists,ignore_index=True)
train_pos_df.head()

,review,target
0,Story of a man who has unnatural feelings for ...,1
1,OK its not the best film I've ever seen but at...,1
2,"Amateur, no budget films can be surprisingly g...",1
3,My girlfriend once brought around The Zombie C...,1
4,"Without wishing to be a killjoy, Brad Sykes is...",1


In [3]:

train_df = pd.concat([ train_pos_df,  train_neg_df ],ignore_index=True)
train_df.head()

,review,target
0,Story of a man who has unnatural feelings for ...,1
1,OK its not the best film I've ever seen but at...,1
2,"Amateur, no budget films can be surprisingly g...",1
3,My girlfriend once brought around The Zombie C...,1
4,"Without wishing to be a killjoy, Brad Sykes is...",1


In [4]:

train_df.to_csv("movie_data.csv", index=False, encoding='utf-8')

In [5]:

df = pd.read_csv('movie_data.csv')
df.head()

,review,target
0,Story of a man who has unnatural feelings for ...,1
1,OK its not the best film I've ever seen but at...,1
2,"Amateur, no budget films can be surprisingly g...",1
3,My girlfriend once brought around The Zombie C...,1
4,"Without wishing to be a killjoy, Brad Sykes is...",1


In [ ]:

# BoW(Bag of Words) 모델
# 문자를 숫자벡터
# 단어의 등장횟수를 카운트

# 전체 훈련데이터에서 모든 고유한 단어(토큰)로 어휘 사전
# 각 문서(리뷰데이터)를 사전을 기준으로 벡터화  N번째단어가 문서에서 3번나오면 벡터의 N번째값이 3이 된다.
# 문서1 : "나는 영화가 좋다"
# 문서2 : "나는 영화가 싫다"
# 사전 : {'나는':0, '영화가':1,'좋다':2,'싫다':3}
# 벡터화는 사전의 크기만큼 모든 문장의 길이를 동일하게
# 문서1벡터 : [0,1,2] - > [1,1,1,0]
# 문서2벡터 : [0,1,3] - > [1,1,0,1]

In [6]:

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array([
    "The sun is shining",
    "The weather is sweet"
])
bag = count.fit_transform(docs)

In [7]:
count.vocabulary_

{'the': 4, 'sun': 2, 'is': 0, 'shining': 1, 'weather': 5, 'sweet': 3}

In [ ]:
# 문서 d에 등장한 단어 t의 횟수를 tf(t,d)
# BoW를 보완하면서 좀더 정교환 텍스트 벡터화 방식  TF-IDF(Term Frequency -Inverse Document Frequency)
# TF : 특정문서에서 자주 등장하는 단어
# IDF : 전체문서에 드물게 등장하는 단어
# 특정문서에서 자주 등장하지만 전체 문장에서 드물게 등장하는 단어에 높은 가중치를 부여 - 그 문장을 잘 대표하는 핵심 단어를 찾는다
# TF(t,d)  단어 t가 문장 d에 나타난 횟수 / 문서d의 모든 단어수
# IDF(t,D) : log( 총 문서수|D| / 단어 t를 포함한 문서의수 df(t) )  -- log 단어의 희귀성을 너무 과하게 반영하지 않도록 스케일링
# 분모에 + 1(사이킷런의 경우) :  분모가 0되는 것을 방지
# log( 1+ |D| / 1+df(f)  )

#TF-IDF(t,d,D) = TF(t,d) x IDF(t,D)

# "나는"
# TF : 리뷰에 3번 나옴 (높음)
# IDF : 전체 10,000개 리뷰중에 9000개 나옴(매우 낮음)
# tf-idf  높음 x 매우낮음 = 낮음(중요도가 낮음)

# "명작" TF : 리뷰에 2번 나옴 (높음)
# IDF : 전체 10,000개 리뷰중에 50개 나옴(매우 높음)
# tf-idf  높음 x 매우높음 = 높음(핵심단어)

In [8]:

df.review[0]

"Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly."

In [9]:

# 데이터 정재... html tag 와 같은 불필요한 string이 보임... 특수기호 기타등등.  < - ' 
import re
def preprocessor(s):
    # 1. 영문, 공백, ., , 만 남기기
    clean = re.sub(r'[^A-Za-z\s.,]+', '', s)
    # 2. 연속된 마침표(...)를 마침표 하나로
    clean = re.sub(r'\.{2,}', '.', clean)
    # 3. 연속된 공백 정리
    clean = re.sub(r'\s+', ' ', clean).strip()
    return clean

In [10]:

df['review'] = df.review.apply(preprocessor)

In [ ]:

# 문서를 토큰으로 나누기
# %pip install nltk
# %conda install -c conda-forge nltk

In [11]:

from nltk.stem.porter import PorterStemmer

def tokenizer(text):
    return text.split()

porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

# 어간추출 Stemming  단어의 접미사 -s -es -ing -ed 등등.. 를 강제로 제거해서 단어의 원형을 찾는과정
tokenizer(df.review[0][:100])

['Story',
 'of',
 'a',
 'man',
 'who',
 'has',
 'unnatural',
 'feelings',
 'for',
 'a',
 'pig.',
 'Starts',
 'out',
 'with',
 'a',
 'opening',
 'scene',
 'that',
 'is',
 'a',
 'terri']

In [12]:

tokenizer('runners like running')

['runners', 'like', 'running']

In [13]:

# 불용어
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')    # 불용어 사전 다운로드
stops =  stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SAMSUNG\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:

df.head()  # 전처리완료  정규식을 이용한..

,review,target
0,Story of a man who has unnatural feelings for ...,1
1,OK its not the best film Ive ever seen but at ...,1
2,"Amateur, no budget films can be surprisingly g...",1
3,My girlfriend once brought around The Zombie C...,1
4,"Without wishing to be a killjoy, Brad Sykes is...",1


In [15]:
from sklearn.model_selection import train_test_split
X = df.review
y = df.target
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=42)

In [16]:

# 어간추출(postStemmer) -> stopwor에 포함된 단어제거
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split() if word not in stops]

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [17]:

tfidf = TfidfVectorizer(
    tokenizer=tokenizer_porter,
    ngram_range=(1,1) # (1,1) 유니그램(unigramm, 단일단어)만 사용
)
pipeline = Pipeline([
    ('tfidf', tfidf),
    ('clf', LogisticRegression())
])

In [18]:
pipeline.fit(x_train,y_train)

c:\Users\SAMSUNG\anaconda3\envs\deep\Lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


,steps,"[('tfidf', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,<function tok...0028886AF9620>


In [19]:
print(x_test.to_numpy()[0])
print(y_test.to_numpy()[0])

This film is really vile. It plays on the urban paranoia of the ss and puts it into a school context. Im not saying that urban crime wasnt a problem for a lot of people or that schools werentarent problem areas but this vile piece of exploitation takes the biscuit. Violence is beyond anything realistically imaginable but in this case its not a case of social issues but a white, uppermiddle class student uses it to turn himself into the crime kingpin of his local high schoiol. And of course he knows how to play the system. Does that sound familiar. Yes. This turd is pure violent exploitation, a really nasty piece of work. Its disturbing brutality dressed up as a social comment. This belongs in the same category as trash like Exterminator, Death Wish and so on and so on. The only remarkable thing is that Michael Fox was so broke at the time that he had to do stuff like this.
1


In [20]:

pipeline.score(x_test,y_test)

0.095

In [ ]:
# 원본데이터로드..
# 토크나이져 함수를 정의
    # 텍스트 전처리
    # 공백을 기준으로 단어단위로 분리
    # 영어는 전부 소문자로 변환
    # 어간 추출
    # 불용어 제거
# TFIDF를 정의
    # 토크나이져 매개변수 = 토크나이져 함수
    # ngram  (1,1)
# 파이프라인으로 tfidf, 머신러닝
# 파이프라이으로 학습
# 파이프라인으로 평가( classification_report)
# 과적합여부 확인

# train 폴더에 있는데이터로 학습 - 적당한 크기로
# test 폴더에 있는 문장으로 평가

In [ ]:

from glob import glob
import numpy as np
from tqdm import tqdm

def get_data(pattern, neg=True,to = None):     # neg : True면 부정(negative) 리뷰, False면 긍정(positive) 리뷰/ to : 처음부터 몇 개의 파일만 읽을지 제한할 수 있음 (없으면 전체 파일 읽음)
    documents = []
    if to is not None:
        for path in tqdm(glob(pattern)[ : to]):     # [:to] 파일을 to 개수만큼만 읽기 (예: [:100] → 앞의 100개만)
            with open(path, 'r', encoding='utf-8') as f:
                documents.append(  (np.array(f.read()),  0 if neg else 1)   )       # f.read() 파일 전체 내용을 문자열로 읽음
    else:                                                                           # (np.array(f.read()), 0 if neg else 1) -> (리뷰 내용, 레이블) 형태의 튜플 생성
        for path in tqdm(glob(pattern)):
            with open(path, 'r', encoding='utf-8') as f:
                documents.append(  (np.array(f.read()),  0 if neg else 1)   )
    return documents

# "../data/movie/train/neg/*.txt"  

In [ ]:
train_neg_lists= get_data('C:/python_src/machine_learning/감성분석/data/movie/train/neg/*.txt',True)        # neg가 True면 라벨 0 넣기
train_pos_lists= get_data('C:/python_src/machine_learning/감성분석/data/movie/train/pos/*.txt',False)       # neg가 Flase면 라벨 1 넣기
test_neg_lists= get_data('C:/python_src/machine_learning/감성분석/data/movie/test/neg/*.txt',True)
test_pos_lists= get_data('C:/python_src/machine_learning/감성분석/data/movie/test/pos/*.txt',False)


100%|██████████| 500/500 [00:00<00:00, 529.56it/s]


In [23]:

train_lists = train_neg_lists + train_pos_lists
test_lists = test_neg_lists + test_pos_lists
len(train_lists), len(test_lists)

(2020, 1000)

In [ ]:
import re
porter = PorterStemmer()
def custom_tokenizer(text):                         # 전처리& 토큰화
    # 전처리
    # 1. 영문, 공백, ., , 만 남기기
    clean = re.sub(r'[^A-Za-z\s.,]+', '', text)
    # 2. 연속된 마침표(...)를 마침표 하나로
    clean = re.sub(r'\.{2,}', '.', clean)
    # 3. 연속된 공백 정리
    clean = re.sub(r'\s+', ' ', clean).strip()
    # 단어분리-어간분리-불용어제거
    return [porter.stem(word) for word in clean.split() if word not in stops]
tfidf = TfidfVectorizer(                    # TF-IDF 벡터화 단계
    tokenizer=custom_tokenizer,             # 우리가 직접 만든 전처리/토큰화 함수를 사용
    ngram_range=(1,1),                      # 단어 1개씩(uni-gram) 단위로 벡터화 (예: “good movie”)
    token_pattern=None                      # 기본 정규식 토큰화 방식을 끔 → 우리가 지정한 tokenizer만 사용
)
pipeline = Pipeline([                       # 파이프라인 구성
    ('tfid',tfidf),
    ('clf',LogisticRegression())
])

In [ ]:
# (리뷰, 라벨) 형태의 리스트 데이터를 → 모델 학습에 쓸 수 있는 NumPy 배열(X, y) 로 변환
x = np.array([str(x.item()) for x,_ in train_lists])
y = np.array([ y for _,y in train_lists])

In [26]:

pipeline.fit(x,y)

,steps,"[('tfid', ...), ('clf', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,<function cus...002888125F060>


In [27]:
x_test = np.array([str(x.item())for x,y in test_lists])
y_test = np.array([y for _,y in test_lists])
pipeline.score(x_test,y_test)


0.793

In [29]:
from sklearn.metrics import classification_report
print( classification_report(y_test, pipeline.predict(x_test)) )

              precision    recall  f1-score   support

           0       0.76      0.85      0.80       500
           1       0.83      0.74      0.78       500

    accuracy                           0.79      1000
   macro avg       0.80      0.79      0.79      1000
weighted avg       0.80      0.79      0.79      1000

